# Voorbeeld van omzetting door Veg2Hab

Dit notebook doorloopt alle stappen die tot nu toe geimplementeerd zijn voor Veg2Hab. Het is een voorbeeld van hoe de omzetting van vegetatie naar habitattypekaart kan verlopen.

Eerst worden alle benodigde resources geinstantieerd (was-wordt lijst, definitietabel, fgr-kaart). Hierna wordt de omzetting stap voor stap uitgevoerd.

In [1]:
from pathlib import Path

import sys
sys.path.append('../')

from veg2hab.waswordtlijst import WasWordtLijst, opschonen_waswordtlijst
from veg2hab.definitietabel import DefinitieTabel, opschonen_definitietabel
from veg2hab.vegkartering import Kartering
import pandas as pd
from veg2hab.fgr import FGR

pd.set_option('display.max_columns', 100)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


### Was-Wordt lijst opschonen en inladen

VvN en SBB codes worden gecheckt op validiteit bij:
1. het opschonen van een was-wordt lijst
2. bij het inlezen van een opgeschoonde was-wordt lijst

In [2]:
path_in_wwl = Path("../data/5. Was-wordt-lijst-vegetatietypen-en-habitattypen-09-02-2021.xlsx")
path_out_wwl = Path("../testing/opgeschoonde_waswordt.xlsx")
opschonen_waswordtlijst(path_in_wwl, path_out_wwl)
wwl = WasWordtLijst.from_excel(path_out_wwl)
wwl.df.head(3)

/home/jordydelange/.cache/pypoetry/virtualenvs/veg2hab-CuqoUkZb-py3.7/lib/python3.7/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Alle SBB codes zijn valide
Alle VvN codes zijn valide
Alle SBB codes zijn valide
Alle VvN codes zijn valide


,VvN,SBB
0,14rg10,14-l
1,None,7-a
2,None,8b-b


### Definitietabel opschonen en inladen

VvN- en SBB-codes worden gecheckt op validiteit bij:
1. het opschonen van een definitietabel
2. bij het inlezen van een opgeschoonde definitietabel

In [3]:
path_in_dt = Path("../data/definitietabel habitattypen (versie 24 maart 2009)_0.xls")
path_in_mitsjson = Path("../data/mitsjson.csv")
path_in_mozaiekjson = Path("../data/mozaiekjson.csv")
path_out_dt = Path("../testing/opgeschoonde_definitietabel.xlsx")
opschonen_definitietabel(path_in_dt, path_in_mitsjson, path_in_mozaiekjson, path_out_dt)
dt = DefinitieTabel.from_excel(path_out_dt)
dt.df.head()

Alle SBB codes zijn valide
Alle VvN codes zijn valide


,DT regel,Habitattype,Kwaliteit,SBB,VvN,mits,mozaiek,mitsjson,mozaiekjson,Criteria,Mozaiekregel
0,2,H1110_A,Kwaliteit.GOED,50a,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
1,3,H1110_A,Kwaliteit.GOED,50b,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
2,4,H1110_A,Kwaliteit.GOED,50c,None,mits in de slikkige en fijnzandige delen van F...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
3,5,H1110_B,Kwaliteit.GOED,50a,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)
4,6,H1110_B,Kwaliteit.GOED,50b,None,mits in de grofzandige delen van FGR Getijdeng...,<NA>,"{ ""type"": ""EnCriteria"", ""sub_criteria""...","{ ""type"" : ""GeenMozaiekregel"" }",((FGR is Getijdengebied of FGR is Noordzee) en...,Geen mozaiekregel (altijd waar)


In [4]:
dt.df.Mozaiekregel.iloc[-6]

StandaardMozaiekregel(mozaiek_threshold=90, habtype='H91E0_C', alleen_zelfstandig=True, alleen_goede_kwaliteit=False, keys=[('H91E0_C', True, <Kwaliteit.GOED: 'Goed'>), ('H91E0_C', True, <Kwaliteit.MATIG: 'Matig'>)], habtype_percentage_dict=None)

### FGR inladen

Bij het inladen worden de FGR typen omgezet naar de FGRType Enum, en hierdoor vind ook validatie plaats.

In [5]:
fgr = FGR(Path("../data/bronbestanden/FGR.json"))
fgr.gdf.head(3)

,fgr,geometry
0,FGRType.DU,"POLYGON ((105147.000 526863.188, 105092.000 52..."
1,FGRType.DU,"POLYGON ((111314.000 552409.000, 111289.008 55..."
2,FGRType.DU,"POLYGON ((119244.333 577690.288, 119230.924 57..."


## Inladen van een kartering vanaf een shapefile

In [6]:
shp_path = Path("../testing/vegetatiekarteringen/GR/SGL Hunzedal en Leekstermeer2021/2021 Vegetatiekartering Leekstermeer2021/GIS bestanden Onlanden 2021/Vegetatiekartering_Leekstermeer2021.shp")
Kartering.from_shapefile(shp_path, "elmid", vegtype_col_format="single", sbb_of_vvn="sbb", SBB_col="SBBTYPE", split_char="+").gdf

,ElmID,geometry,SBBTYPE,SBBTYPE1,SBBTYPE2,SBBTYPE3,Datum,Opmerking,Opp,_LokVrtNar,perc_0,perc_1,perc_2,VegTypeInfo
0,144,"POLYGON ((225536.382 578550.118, 225536.613 57...",08C-a+16-a,8c-a,16-a,None,None,None,1692.139818,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8c-a'], VvN: []), (50.0%, SBB:..."
1,143,"POLYGON ((225518.346 578389.608, 225514.596 57...",08-a+08C2c,8-a,8c2c,None,None,None,2974.448552,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8-a'], VvN: []), (50.0%, SBB: ..."
2,73,"POLYGON ((225398.991 578372.192, 225399.792 57...",16-a,16-a,None,None,None,None,4582.870987,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['16-a'], VvN: [])]"
3,62,"POLYGON ((225338.120 578506.385, 225330.908 57...",12B1d+50A,12b1d,50a,None,None,None,2843.780077,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['12b1d'], VvN: []), (50.0%, SBB..."
4,111,"POLYGON ((225731.271 578748.007, 225730.075 57...",50A+08B3d,50a,8b3d,None,None,None,1911.539469,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['50a'], VvN: []), (50.0%, SBB: ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,193,"POLYGON ((225335.414 578905.115, 225334.448 57...",12B1d,12b1d,None,None,None,None,442.032701,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['12b1d'], VvN: [])]"
267,220,"POLYGON ((225121.987 578757.930, 225125.003 57...",08-k,8-k,None,None,None,None,361.533352,Lokale typologie is primair vertaald naar sbb,100.0,0.0,0.0,"[(100.0%, SBB: ['8-k'], VvN: [])]"
268,58,"POLYGON ((225297.768 578230.760, 225292.476 57...",08B3a+08C-f,8b3a,8c-f,None,None,None,722.439797,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['8b3a'], VvN: []), (50.0%, SBB:..."
269,249,"POLYGON ((224515.275 578618.372, 224510.014 57...",39A1c+08B3a,39a1c,8b3a,None,None,None,713.229564,Lokale typologie is primair vertaald naar sbb,50.0,50.0,0.0,"[(50.0%, SBB: ['39a1c'], VvN: []), (50.0%, SBB..."


## Inladen en omzetten van een access kartering

- Voor ieder vlak in de kartering wordt de (complexe) SBB code opgezocht.
- Deze SBB-codes worden opgeschoond en gevalideerd.
- De SBB codes worden toegevoegd aan de dataframe van de kartering als VegTypeInfo instance.
  
De resulterende dataframe heeft voor elke regel in KarteringVegetatietype.csv een VegTypeInfo instance in de rij van de betreffende shape.

In [7]:
shape_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/vlakken.shp")
csvs_path = Path("../data/notebook_data/Rottige_Meenthe_Brandemeer_2013/864_RottigeMeenthe2013/")
shape_elm_id_column = "ElmID"

access_kartering = Kartering.from_access_db(shape_path, shape_elm_id_column, csvs_path)

access_kartering.gdf.head(3)


,ElmID,Opmerking,Datum,geometry,Opp,_LokVrtNar,intern_id,Locatie,VegTypeInfo
0,28049,None,19072013,"POLYGON ((189433.818 538314.302, 189439.215 53...",3761.736281,Lokale typologie is primair vertaald naar SBB,1809,1809,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9..."
1,15034,None,10092013,"POLYGON ((190568.500 539171.122, 190538.023 53...",10442.658431,Lokale typologie is primair vertaald naar SBB,815,815,"[(100%, SBB: ['50a'], VvN: [])]"
2,13503,None,14102013,"POLYGON ((189538.104 539409.332, 189532.399 53...",2174.098680,Lokale typologie is primair vertaald naar SBB,650,650,"[(90%, SBB: ['9a2a'], VvN: []), (10%, SBB: ['9..."


In [8]:
# Oke even voor testen

shape_path = Path("../testing/vegetatiekarteringen/GR/SBB ZWK 2010/0814_Tussen de Gasten 2010/ZWK0814_2010.shp")
csvs_path = Path("../testing/vegetatiekarteringen/DR Extra/NM_DCR2013/DCR2013_DigitaleStandaard/DCR fochteloo")

shape_elm_id_column = "OBJECTID"

# Elmid	Datum	Opmerking	single	SBB	SBBTYPE		+

access_kartering = Kartering.from_shapefile(
    shape_path, "Elmid", "single", "SBB", None, None, "SBBTYPE", None, "+", None
)

access_kartering.gdf.head(3)

g = access_kartering.gdf

In [9]:
g[g.ElmID == 2275]

,ElmID,geometry,SBBTYPE,SBBTYPE1,SBBTYPE2,SBBTYPE3,Datum,Opmerking,Opp,_LokVrtNar,perc_0,perc_1,perc_2,VegTypeInfo
1274,2275,"POLYGON ((211164.843 581119.667, 211156.246 58...",16A-c+16A-c,16a-c,16a-c,None,None,None,1714.39835,Lokale typologie is primair vertaald naar SBB,50.0,50.0,0.0,"[(50.0%, SBB: ['16a-c'], VvN: []), (50.0%, SBB..."


#### Opzoeken en toevoegen VvN aan kartering

In [10]:
access_kartering.apply_wwl(wwl)
# Voorbeeld van 3 vegtypeinfos in 1 geometry
access_kartering.gdf["VegTypeInfo"].iloc[3]

[VegTypeInfo(percentage=100.0, SBB=[SBB(klasse='9', verbond='a', associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='a')], VvN=[VvN(klasse='9', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='2')])]

#### Vinden van de mogelijke habitattypen voor een VegTypeInfo

Voor elke VegTypeInfo worden de mogelijke habitattypen opgezocht in de definitietabel. Deze worden vervolgens in een HabitatVoorstel gezet, samen met de code waarop de match is gemaakt, de regel in de definitietabel die is gebruikt en het niveau van de match.

In [11]:
access_kartering.apply_deftabel(dt)
access_kartering.gdf["HabitatVoorstel"]

0       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
1       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
2       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
3       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
4       [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
                              ...                        
2002    [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
2003    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
2004    [[HabitatVoorstel(onderbouwend_vegtype=SBB(kla...
2005    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
2006    [[HabitatVoorstel(onderbouwend_vegtype=VvN(kla...
Name: HabitatVoorstel, Length: 2007, dtype: object

In [12]:
# Voorbeeld met voor 2 VegTypeInfo's waarbij de eerste twee mogelijke voorstellen heeft
access_kartering.gdf["HabitatVoorstel"][45]

[[HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='16', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='8'), vegtype_in_dt=None, habtype='H0000', kwaliteit=<Kwaliteit.NVT: 'Nvt'>, idx_in_dt=None, mits=GeenCriterium(), mozaiek=GeenMozaiekregel(mozaiek_threshold=90), match_level=<MatchLevel.NO_MATCH: 0>, mozaiek_dict=None)],
 [HabitatVoorstel(onderbouwend_vegtype=VvN(klasse='12', orde=None, verbond=None, associatie=None, subassociatie=None, derivaatgemeenschap=None, rompgemeenschap='3'), vegtype_in_dt=None, habtype='H0000', kwaliteit=<Kwaliteit.NVT: 'Nvt'>, idx_in_dt=None, mits=GeenCriterium(), mozaiek=GeenMozaiekregel(mozaiek_threshold=90), match_level=<MatchLevel.NO_MATCH: 0>, mozaiek_dict=None)]]

### De mitsen en mozaiekregels checken en de HabitatVoorstellen omzetten naar HabitatKeuzes

In [13]:
access_kartering.bepaal_habitatkeuzes(fgr)

Iteratie 0: van 2342 naar 62 keuzes nog te bepalen
Iteratie 1: van 62 naar 14 keuzes nog te bepalen
Iteratie 2: van 14 naar 13 keuzes nog te bepalen
Iteratie 3: van 13 naar 13 keuzes nog te bepalen
0
0


/home/jordydelange/git/veg2hab/veg2hab/vegkartering.py:1076: UserWarning: Er zijn nog 13 habitatkeuzes die niet bepaald konden worden.
  f"Er zijn nog {n_keuzes_still_to_determine_post} habitatkeuzes die niet bepaald konden worden."


### De kartering formatten als een Habitattypenkartering

Dit pakt de HabitatVoorstellen uit en format het geheel als een Habitattypenkartering zoals beschreven in Gegevens Leverings Protocol (uitvraag bijlage 3a).

Voor nu checken we nog geen mitsen en mozaiek en wordt voor ieder VegTypeInfo het eerste HabitatVoorstel gebruikt.

In [14]:
final_format = access_kartering.as_final_format()
final_format

,Area,Opm,Datum,ElmID,geometry,_Samnvttng,_LokVrtNar,Habtype1,Perc1,Opp1,Kwal1,Opm1,VvN1,SBB1,_Status1,_Uitleg1,_VgTypInf1,_Mits_opm1,_Mozk_opm1,_MozkDict1,_VvNdftbl1,_SBBdftbl1,Habtype2,Perc2,Opp2,Kwal2,Opm2,VvN2,SBB2,_Status2,_Uitleg2,_VgTypInf2,_Mits_opm2,_Mozk_opm2,_MozkDict2,_VvNdftbl2,_SBBdftbl2,Habtype3,Perc3,Opp3,Kwal3,Opm3,VvN3,SBB3,_Status3,_Uitleg3,_VgTypInf3,_Mits_opm3,_Mozk_opm3,_MozkDict3,_VvNdftbl3,_SBBdftbl3
0,334.013771,None,None,1001,"POLYGON ((215888.890 575807.799, 215883.177 57...",100.0% HXXXX,Lokale typologie is primair vertaald naar SBB,HXXXX,100,33401.4,X,Er zijn mitsen of mozaiekregels die nog niet g...,16ab4,None,KeuzeStatus.PLACEHOLDER,Er zijn placeholder mitsen/mozaiekregels gevon...,"(100.0%, SBB: ['16b1a'], VvN: ['16ab4'])","Alle niet false mitsen: [['16ab4', 'H7230', 'P...","Alle niet false mozaiekregels: [['16ab4', 'H72...",,"['16a', 589]",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9826.949562,None,None,1002,"POLYGON ((214861.593 576116.124, 214820.543 57...",100.0% H0000,Lokale typologie is primair vertaald naar SBB,H0000,100,982695,X,Geen van de opgegeven vegetatietypen is terugg...,12rg3,None,KeuzeStatus.VEGTYPEN_NIET_IN_DEFTABEL,De vegetatietypen van het vlak zijn niet in de...,"(100.0%, SBB: ['16-k'], VvN: ['12rg3'])",,,,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16982.902956,None,None,1003,"POLYGON ((215459.957 576191.918, 215473.144 57...",100.0% H0000,Lokale typologie is primair vertaald naar SBB,H0000,100,1.69829e+06,X,Geen van de opgegeven vegetatietypen is terugg...,12ba1b,None,KeuzeStatus.VEGTYPEN_NIET_IN_DEFTABEL,De vegetatietypen van het vlak zijn niet in de...,"(100.0%, SBB: ['12b1a'], VvN: ['12ba1b'])",,,,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,885.541572,None,None,1004,"POLYGON ((211921.050 576390.502, 211923.780 57...",100.0% H0000,Lokale typologie is primair vertaald naar SBB,H0000,100.0,88554.15718527493,X,Er zijn geen habitatvoorstellen waarvan zowel ...,"['9rg2', '9rg2', '9rg2', '9rg2']","[None, None, None, None]",KeuzeStatus.GEEN_KLOPPENDE_MITSEN,Er is geen habitatvoorstel met kloppende mits/...,"(100.0%, SBB: ['9a-a'], VvN: ['9rg2'])",Mitsen waaraan mogelijk niet is voldaan: [['9r...,Mozaiekregels waaraan mogelijk niet is voldaan...,"51.79% zelfstandig H0000, 48.21% zelfstandig H...","[""['9rg2', 229]"", ""['9rg2', 558]"", ""['9rg2', 5...","[None, None, None, None]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3942.579119,None,None,1005,"POLYGON ((216185.690 576405.307, 216186.285 57...",100.0% H0000,Lokale typologie is primair vertaald naar SBB,H0000,100,394258,X,Geen van de opgegeven vegetatietypen is terugg...,8bc4,None,KeuzeStatus.VEGTYPEN_NIET_IN_DEFTABEL,De vegetatietypen van het vlak zijn niet in de...,"(100.0%, SBB: ['8c4a'], VvN: ['8bc4'])",,,,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,269.358208,None,None,3003,"POLYGON ((214570.433 576552.086, 214566.692 57...",100.0% HXXXX,Lokale typologie is primair vertaald naar SBB,HXXXX,100.0,26935.820842579567,X,Er zijn mitsen of mozaiekregels die nog niet g...,"[None, None]","['50a', '50a']",KeuzeStatus.PLACEHOLDER,Er zijn placeholder mitsen/mozaiekregels gevon...,"(100.0%, SBB: ['50a'], VvN: [])","Alle niet false mitsen: [['50a', 'H7110_A', 'P...","Alle niet false mozaiekregels: [['50a', 'H7110...",85.96% zelfstandig HXXXX,"[None, None]","[""['50a', 478]"", ""['50a', 585]""]",NaN,NaN,Na

In [15]:
access_kartering.final_format_to_file(Path("../testing/veg2hab_output.shp"))